In [1]:
import pandas as pd
import numpy as np

In [2]:
import pickle

In [3]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [4]:
from sklearn.metrics import auc, roc_auc_score, roc_curve, accuracy_score,  recall_score, precision_score
from regression_tools.dftransformers import (
    ColumnSelector, Identity, FeatureUnion, MapFeature, Intercept)

In [5]:
from my_metr import plot_roc_curve

In [6]:
from trans_rat import FillPrice, CategoriesTransformScr, RatingNormalizer, FillPriceScr


In [7]:
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
from pymongo import MongoClient
import time
from bs4 import BeautifulSoup

mc = MongoClient()
db  = mc['raw_restaurants']
rv_s = db['reviews_scrap']

In [9]:
colClust_n = ['cl_t0', 'cl_t1', 'cl_t3', 'cl_t4', 'cl_t5', 'cl_t8', 'cl_t9']

In [ ]:
#numbers_list = pickle.load(list_unpickle)

In [10]:
clust_model_rf ={}
clust_model_gr = {}

In [11]:
for col in colClust_n:
    filename_gr = 'gr10boost/mgr_'+col+'.pkl'
    with open(filename_gr,'rb') as f:
        clust_model_gr[col]= pickle.load( f)
    filename_rf = 'r10forest/mrf_'+col+'.pkl'
    with open(filename_rf,'rb') as f:
        clust_model_rf[col] =pickle.load( f)

In [12]:
clust_model_gr['cl_t3']

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=6,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=300,
              presort='auto', random_state=179, subsample=0.8, verbose=0,
              warm_start=False)

In [14]:
user_cl = pd.read_csv('user_from_scr_clidf10.csv')

In [15]:
rest = db['restaurants']

rest_df = pd.DataFrame(list(rest.find({})))

In [16]:
rest_df.shape

(12615, 18)

### надо почистить дату

In [17]:
revs_df = pd.DataFrame(list(rv_s.find({})))

In [18]:
revs_df.columns

Index(['_id', 'alias', 'biz_id', 'category', 'date', 'id', 'rating', 'text',
       'user_id'],
      dtype='object')

In [19]:
revs_df.drop(columns=[ '_id', 'category', 'date'], inplace= True)

In [20]:
rest_alias = np.unique(revs_df.alias)

In [21]:
len(rest_alias)

10222

In [22]:
rest_df.head(3)

,_id,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,rev_API,review_count,transactions,url
0,5bf90d4e7988471b045ba501,maono-seattle,"[{'alias': 'hawaiian', 'title': 'Hawaiian'}, {...","{'latitude': 47.5636486287425, 'longitude': -1...",(206) 935-1075,1501.527536,gDAO-4m63svPUpii5b8kww,https://s3-media4.fl.yelpcdn.com/bphoto/rwJG59...,False,"{'address1': '4437 California Ave SW', 'addres...",Ma'ono,+12069351075,$$,4.0,1,788,[],https://www.yelp.com/biz/maono-seattle?adjust_...
1,5bf90d4e7988471b045ba502,ohs-sandwiches-seattle-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...","{'latitude': 47.5751165, 'longitude': -122.387...",(206) 420-3692,2616.713742,xzLAynrVb3No3ADUhMxlnQ,https://s3-media2.fl.yelpcdn.com/bphoto/zKlK95...,False,"{'address1': '3217 California Ave SW', 'addres...",Oh's Sandwiches,+12064203692,NaN,4.5,1,32,[],https://www.yelp.com/biz/ohs-sandwiches-seattl...
2,5bf90d4e7988471b045ba503,marination-ma-kai-seattle,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...","{'latitude': 47.5895547295386, 'longitude': -1...",(206) 328-8226,4061.556690,IsvlaFF0EPEhh3ahTuC4xg,https://s3-media1.fl.yelpcdn.com/bphoto/r19fwm...,False,"{'address1': '1660 Harbor Ave SW', 'address2':...",Marination Ma Kai,+12063288226,$$,4.0,1,897,[],https://www.yelp.com/biz/marination-ma-kai-sea...


In [23]:
rest_clean = rest_df.drop(columns= ['_id','coordinates', 'display_phone', 
                                    'distance','image_url', 'name','location',
                                   'is_closed','phone', 'transactions','url'])

In [24]:
rest_clean.head(3)

,alias,categories,id,price,rating,rev_API,review_count
0,maono-seattle,"[{'alias': 'hawaiian', 'title': 'Hawaiian'}, {...",gDAO-4m63svPUpii5b8kww,$$,4.0,1,788
1,ohs-sandwiches-seattle-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",xzLAynrVb3No3ADUhMxlnQ,NaN,4.5,1,32
2,marination-ma-kai-seattle,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",IsvlaFF0EPEhh3ahTuC4xg,$$,4.0,1,897


In [35]:
#Pipeline = Pipeline([FillPriceScr, CategoriesTransformScr, RatingNormalizer ])


In [25]:
fpr = FillPriceScr()

In [26]:
rest_cl2 = fpr.transform(rest_clean)
rest_cl2.head(2)

/Users/olga/yelper/trans_rat.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  X[self.priceCol][pd.isnull(X[self.priceCol])] = '$$'


,alias,categories,id,RestaurantsPriceRange2,rating,rev_API,review_count,rest_isna
0,maono-seattle,"[{'alias': 'hawaiian', 'title': 'Hawaiian'}, {...",gDAO-4m63svPUpii5b8kww,2,4.0,1,788,False
1,ohs-sandwiches-seattle-2,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",xzLAynrVb3No3ADUhMxlnQ,2,4.5,1,32,True


In [27]:
##Acai Bowls
mod_cat = pd.read_csv('model_cat.csv')
mod_cat.head(3)

,Unnamed: 0,0
0,Acai Bowls,0.000840
1,Active Life,0.004093
2,Afghan,0.002064


In [28]:
mod_fe = mod_cat.iloc[:,0].tolist()
rest_cl2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12615 entries, 0 to 12614
Data columns (total 8 columns):
alias                     12615 non-null object
categories                12615 non-null object
id                        12615 non-null object
RestaurantsPriceRange2    12615 non-null int64
rating                    12615 non-null float64
rev_API                   8945 non-null object
review_count              12615 non-null int64
rest_isna                 12615 non-null bool
dtypes: bool(1), float64(1), int64(2), object(4)
memory usage: 702.3+ KB


In [29]:
ct = CategoriesTransformScr(mod_fe)
rn = RatingNormalizer(starCol = 'rating')
rest_cx = rn.transform(ct.transform(rest_cl2))
rest_cx.head()

,alias,id,RestaurantsPriceRange2,stars,rev_API,review_count,rest_isna,Acai Bowls,Active Life,Afghan,...,Venues & Event Spaces,Vietnamese,Waffles,Whiskey Bars,Wine & Spirits,Wine Bars,Wineries,Wraps,num_of_categ,st_over_am
0,maono-seattle,gDAO-4m63svPUpii5b8kww,2,4.0,1,788,False,False,False,False,...,False,False,False,False,False,False,False,False,2,0.016000
1,ohs-sandwiches-seattle-2,xzLAynrVb3No3ADUhMxlnQ,2,4.5,1,32,True,False,False,False,...,False,False,False,False,False,False,False,False,2,0.140621
2,marination-ma-kai-seattle,IsvlaFF0EPEhh3ahTuC4xg,2,4.0,1,897,False,False,False,False,...,False,False,False,False,False,False,False,False,3,0.016000
3,ittos-tapas-seattle-4,Dr9CxgdePozHGwU4beaYGQ,2,4.5,1,221,False,False,False,False,...,False,False,False,False,False,False,False,False,3,0.020362
4,pizzeria-credo-seattle,BeGo3TSuhF7fOv0dzp0s1g,2,4.5,1,405,False,False,False,False,...,False,False,False,False,False,False,False,False,2,0.018000


In [30]:
### i forgot to include review_count to model when train
colPred1 = rest_cx.columns[2:4].union( rest_cx.columns[6:])

In [32]:
with open('colPred213', 'rb') as f:
    colPred = pickle.load(f)

In [31]:
len(colPred1)

213

In [33]:
colPred1[colPred != colPred1]

Index([], dtype='object')

In [34]:
colPred1

Index(['Acai Bowls', 'Active Life', 'Afghan', 'African', 'American (New)',
       'American (Traditional)', 'Arabian', 'Arcades', 'Argentine',
       'Art Galleries',
       ...
       'Waffles', 'Whiskey Bars', 'Wine & Spirits', 'Wine Bars', 'Wineries',
       'Wraps', 'num_of_categ', 'rest_isna', 'st_over_am', 'stars'],
      dtype='object', length=213)

In [34]:
X = rest_cx[colPred]

,Acai Bowls,Active Life,Afghan,African,American (New),American (Traditional),Arabian,Arcades,Argentine,Art Galleries,...,Wine & Spirits,Wine Bars,Wineries,Wraps,num_of_categ,price,rest_isna,review_count,st_over_am,stars


In [101]:
clt0 = clust_model_rf['cl_t0'].predict_proba(X)[:,1]

In [102]:
clt0

array([0.67499515, 0.2146956 , 0.71224975, ..., 0.21640103, 0.15179362,
       0.42946443])

In [85]:
r0 = clust_model_rf['cl_t0']

## Restaurants Prediction
make 2 dictionary, then 2 dataframe with cluster predictions

In [35]:
%%time
Y_rf = pd.DataFrame()
Y_gr = pd.DataFrame()
Y_rf['alias'] = rest_cx['alias']
Y_gr['alias'] = rest_cx['alias']
for col in colClust_n:
    Y_rf[col] = clust_model_rf[col].predict_proba(X)[:,1]
    Y_gr[col] = clust_model_gr[col].predict_proba(X)[:,1]


CPU times: user 9.07 s, sys: 629 ms, total: 9.7 s
Wall time: 5.27 s


In [36]:
Y_gr.head(3)

,alias,cl_t0,cl_t1,cl_t3,cl_t4,cl_t5,cl_t8,cl_t9
0,maono-seattle,0.013198,0.983233,0.568952,0.125708,0.927498,0.307721,0.138859
1,ohs-sandwiches-seattle-2,0.003599,0.292209,0.029405,0.000707,0.064002,0.004664,0.005442
2,marination-ma-kai-seattle,0.007603,0.983881,0.643542,0.064276,0.924078,0.320205,0.064203


In [37]:
Y_gr['rating'] = rest_cx.stars
Y_rf['rating'] = rest_cx.stars


In [38]:
user_cl.head()

,Unnamed: 0,user_id,user_cl
0,0,SqPqSd7ql_UD5cYxRg_prA,8
1,1,zriuvFjGFSjvER9OgFJCew,6
2,2,havBggvuzfE6IPfkT-njbg,1
3,3,Y5b286tescycns--SAZ-yg,6
4,4,zxn9ZlFJKTTddjF6Du7rRA,0


In [40]:
user_cl.shape, user_cl.columns


((385, 3), Index(['Unnamed: 0', 'user_id', 'user_cl'], dtype='object'))

In [39]:
for col in colClust_n:
    i = int(col[4:])
    u_col = 'u'+col
    user_cl[u_col] = (user_cl['user_cl']==i)
 

In [40]:
user_cl.head()

,Unnamed: 0,user_id,user_cl,ucl_t0,ucl_t1,ucl_t3,ucl_t4,ucl_t5,ucl_t8,ucl_t9
0,0,SqPqSd7ql_UD5cYxRg_prA,8,False,False,False,False,False,True,False
1,1,zriuvFjGFSjvER9OgFJCew,6,False,False,False,False,False,False,False
2,2,havBggvuzfE6IPfkT-njbg,1,False,True,False,False,False,False,False
3,3,Y5b286tescycns--SAZ-yg,6,False,False,False,False,False,False,False
4,4,zxn9ZlFJKTTddjF6Du7rRA,0,True,False,False,False,False,False,False


In [41]:
   
user_cl_n = user_cl.drop(columns=['Unnamed: 0','user_cl'])
user_cl_n = user_cl_n.groupby('user_id').sum()
user_cl_n.head()

,ucl_t0,ucl_t1,ucl_t3,ucl_t4,ucl_t5,ucl_t8,ucl_t9
user_id,,,,,,,
-KlnVOvjklQn5OpTbxFGzA,False,True,False,False,False,False,False
-Yb1srF4RYtpn-6Yp8ojRg,True,True,True,False,False,False,True
1SVDSLiWxbWmBjenfDQkbA,False,True,False,False,True,False,False
1Vq3xObhRXBU8gnPIN9jsQ,False,True,False,False,False,False,False
2wdhN8Y2EJRswvkduFxnOQ,False,True,False,False,False,True,False


In [44]:
user_cl_n.shape

(99, 16)

In [42]:
revs_df.head()

,alias,biz_id,id,rating,text,user_id
0,srivilai-thai-cuisine-seattle,NaN,ErsYo_9v8--feFCy92tymQ,5.0,Fantastic Thai food with warm service. Authent...,_3KTfz0hcPl_yq4jOyOPiA
1,ohs-sandwiches-seattle-2,NaN,W1NnSItt-SwWHrzkRtZ44g,5.0,Fantastic Vietnamese sandwich: great flavors a...,_3KTfz0hcPl_yq4jOyOPiA
2,saison-san-francisco-2,NaN,3Q5uzROHYl1i2B_t2sIl7w,3.0,A fine dining restaurant that is in the midst ...,_3KTfz0hcPl_yq4jOyOPiA
3,international-smoke-san-francisco-2,NaN,-GYe-Aq7NcNJ3u-U6fmwMg,5.0,Great restaurant with an interesting mix of in...,_3KTfz0hcPl_yq4jOyOPiA
4,supercuts-san-francisco,NaN,pPlsW81ntC2qbF0QOe6pLQ,5.0,We always have a good experience here with Ani...,_3KTfz0hcPl_yq4jOyOPiA


In [43]:
rev_user = revs_df.join(user_cl_n, on = 'user_id', how = 'right')

In [44]:
rev_user.shape

(12563, 13)

In [48]:
rr = np.unique(rev_user.alias)
with open('new_r', 'wb') as f:
    pickle.dump(rr,f)

In [45]:
rev_user_biz_gr = rev_user.merge(Y_gr, on = 'alias')

In [46]:
rev_user_biz_rf = rev_user.merge(Y_rf, on = 'alias')

In [47]:
rev_user_biz_gr.shape

(4448, 21)

In [48]:
rev_user_biz_gr.head(3)

,alias,biz_id,id,rating_x,text,user_id,ucl_t0,ucl_t1,ucl_t3,ucl_t4,...,ucl_t8,ucl_t9,cl_t0,cl_t1,cl_t3,cl_t4,cl_t5,cl_t8,cl_t9,rating_y
0,srivilai-thai-cuisine-seattle,NaN,ErsYo_9v8--feFCy92tymQ,5.0,Fantastic Thai food with warm service. Authent...,_3KTfz0hcPl_yq4jOyOPiA,False,True,False,False,...,False,False,0.002571,0.959184,0.120192,0.005282,0.497195,0.029079,0.013973,4.5
1,srivilai-thai-cuisine-seattle,iJjKL0KxMZJCb8MmAsERvA,_oJOAI1Agf5uSbQrT6gS0A,5.0,Today is my 3rd visit and I'm pleased. The di...,iJjKL0KxMZJCb8MmAsERvA,False,True,False,False,...,False,True,0.002571,0.959184,0.120192,0.005282,0.497195,0.029079,0.013973,4.5
2,srivilai-thai-cuisine-seattle,3DLg6FHxYaueFO-47can0w,OaaR9v40Id6czlxw6vicTg,4.0,My boyfriend and I came here last week to cele...,3DLg6FHxYaueFO-47can0w,False,True,False,False,...,False,False,0.002571,0.959184,0.120192,0.005282,0.497195,0.029079,0.013973,4.5


In [49]:
with open('thres_1072_gr', 'rb') as f:
    th_gr72 = pickle.load(f)
with open('thres_1072_rf','rb') as f:
    th_rf72 = pickle.load(f )

In [50]:
with open('thres_1035_gr', 'rb') as f:
    th_gr35 = pickle.load(f)
with open('thres_1035_rf','rb') as f:
    th_rf35 = pickle.load(f )

In [51]:
def predict_rating(row, colClust_n, thres):
    r = False
    for col in colClust_n:
        u_col = 'u'+col
        if row[u_col] and row[col] >= thres[col]:
            r = True
    return r

In [52]:
pred_gr = rev_user_biz_gr.apply(predict_rating, axis=1, args = (colClust_n, th_gr35))

In [53]:
pred_rf = rev_user_biz_rf.apply(predict_rating, axis=1, args = (colClust_n, th_rf35))

In [54]:
act = (rev_user_biz_gr['rating_x'].astype(float) <3)
act_plus = (rev_user_biz_gr['rating_x'].astype(float) <4)

In [55]:
base35 = (rev_user_biz_gr['rating_y'].astype(float) <3.5)
base4 = (rev_user_biz_gr['rating_y'].astype(float) <4)
base45 = (rev_user_biz_gr['rating_y'].astype(float) <4.5)
base5 = (rev_user_biz_gr['rating_y'].astype(float) <5)


In [56]:
recall_score(act, base4), accuracy_score(act, base4), precision_score(act, base4)

(0.596252129471891, 0.6827787769784173, 0.22965879265091863)

In [57]:
recall_score(act, base35), recall_score(act, pred_gr), recall_score(act, pred_rf)

(0.25383304940374785, 0.8620102214650767, 0.7206132879045997)

In [58]:
accuracy_score(act, base35), accuracy_score(act, pred_gr), accuracy_score(act, pred_rf)

(0.8235161870503597, 0.3356564748201439, 0.4435701438848921)

In [59]:
precision_score(act, base35), precision_score(act, pred_gr), precision_score(act, pred_rf)

(0.3004032258064516, 0.14970414201183432, 0.1547183613752743)

In [60]:
from sklearn.metrics import confusion_matrix

In [61]:
confusion_matrix(act, base35)

array([[3514,  347],
       [ 438,  149]])

In [62]:
confusion_matrix(act, base4)

array([[2687, 1174],
       [ 237,  350]])

In [63]:
confusion_matrix(act, base45)

array([[ 980, 2881],
       [  40,  547]])

In [64]:
confusion_matrix(act, pred_gr)

array([[ 987, 2874],
       [  81,  506]])

In [65]:
confusion_matrix(act, pred_rf)

array([[1550, 2311],
       [ 164,  423]])

In [66]:
confusion_matrix(act_plus, pred_rf)

array([[1325, 1711],
       [ 389, 1023]])

In [67]:
confusion_matrix(act_plus, base4)

array([[2214,  822],
       [ 710,  702]])

### Summary after validation on real data

Random Forest models catch bad restaraunts better than base 35, base4, 
and in same time missed less really good restaraunts than base 4
    
With thresholds corresponding true positive rate 35 

In [238]:
rev_user_biz_gr['rating_x'].astype(float).mean(), rev_user_biz_gr['rating_x'].astype(float).std()

(3.778758620689655, 1.1139002849742248)

array([[False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False]])

In [259]:
i = np.random.choice(np.array([0, 1,2]), (4,3))

In [260]:
a = np.array([True, False, True])

In [261]:
a[i]

array([[ True,  True, False],
       [ True,  True, False],
       [ True, False,  True],
       [ True,  True, False]])

### Have to calculate that result is not random

??? bootstrap

In [68]:
from numpy import sqrt
import scipy.stats as scs


def z_test(ctr_old, ctr_new, nobs_old, nobs_new,
           effect_size=0., two_tailed=True, alpha=.05):
    """Perform z-test to compare two proprtions (e.g., click through rates (ctr)).

        Note: if you set two_tailed=False, z_test assumes H_A is that the effect is
        non-negative, so the p-value is computed based on the weight in the upper tail.

        Arguments:
            ctr_old (float):    baseline proportion (ctr)
            ctr_new (float):    new proportion
            nobs_old (int):     number of observations in baseline sample
            nobs_new (int):     number of observations in new sample
            effect_size (float):    size of effect
            two_tailed (bool):  True to use two-tailed test; False to use one-sided test
                                where alternative hypothesis if that effect_size is non-negative
            alpha (float):      significance level

        Returns:
            z-score, p-value, and whether to reject the null hypothesis
    """
    conversion = (ctr_old * nobs_old + ctr_new * nobs_new) / \
                 (nobs_old + nobs_new)

    se = sqrt(conversion * (1 - conversion) * (1 / nobs_old + 1 / nobs_new))

    z_score = (ctr_new - ctr_old - effect_size) / se

    if two_tailed:
        p_val = (1 - scs.norm.cdf(abs(z_score))) * 2
    else:
        # Because H_A: estimated effect_size > effect_size
        p_val = 1 - scs.norm.cdf(z_score)

    reject_null = p_val < alpha
    print('z-score: %s, p-value: %s, reject null: %s' % (z_score, p_val, reject_null))
    return z_score, p_val, reject_null

In [74]:
z_test(0.596252129471891,0.7819420783645656,587,587, effect_size=0.1, two_tailed=False)

z-score: 3.1716214069866235, p-value: 0.0007579524466680976, reject null: True


(3.1716214069866235, 0.0007579524466680976, True)

In [72]:
act.sum()

587